# Create and Train Mechanistic Neural or AMN models

This notebook creates and trains Artificial Metabolic Networks (**AMNs**) that reproduce the behavior of strain models based on simulations or experimental data. The networks compute reaction rates (fluxes) at steady state for either all reactions (fluxes) or a predefined of set experimentally measured or simulated fluxes. Various encoding are proposed composed of a trainable ANN and/or non trainable mechanistic layer:

(1) MM_QP: Use a non trainable mechanistic layer (a gradient descent that minimize the difference between measured and predicted fluxes and minimizes constraints)

(2) ANN_Dense: Use a black box trainable standard ANN to reproduce measured fluxes

(3) AMNs: three models are provided
- AMN_QP: Use trainable ANN and non trainable mechanistic layerto enforce SV = 0 and minimize the different between measured and predicted fluxes. The trainablme layer compute an initial flux vector, fed to teh mechanistci layer. Two programs are available: for extact intake medium fluxes (EB program), and for upper values for intake medium fluxes (UB program)
- AMN_LP: Similar to AMN_QP but using an RNN method found in Y. Yang et al. Mathematics and Computers in Simulation, 101 (2014) 103–112
- AMN_Wt: Use a trainable RNN cell where reaction fluxes are computed with a weight matrix works only when upper values for intake medium fluxes (UB program) are provided

(3) RC: The AMNs are used as non trainable reservoirs to find relationships between nutrient concentration and nutrient uptake fluxes.


# Install conda on your Colab environment

Ignore this first cell if you are running the notebook in a local environment.

One can still run them locally but it will have no effect.

In [1]:
# Run this cell first - it will install a conda distribution (mamba)
# on your Drive then restart the kernel automatically 
# (don't worry about the crashing/restarting kernel messages)
# It HAS to be runned FIRST everytime you use the notebook in colab

import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()

# Set up your Colab or local environment
# Then import libraries

Run this cell in both cases of use (local or Colab)

In [4]:
import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    
    # Check everything is fine with conda in Colab
    import condacolab
    condacolab.check()
    
    # Mount your drive environment in the colab runtime
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    
    # Change this variable to your path on Google Drive to which the repo has been cloned
    # If you followed the colab notebook 'repo_cloning.ipynb', nothing to change here
    repo_path_in_drive = '/content/drive/My Drive/Github/amn_release/'
    # Change directory to your repo cloned in your drive
    DIRECTORY = repo_path_in_drive
    os.chdir(repo_path_in_drive)
    # Copy the environment given in the environment_amn.yml
    !mamba env update -n base -f environment_amn.yml
    
    # This is one of the few Colab-compatible font
    font = 'Liberation Sans'
    
else:
    
    # In this case the local root of the repo is our working directory
    DIRECTORY = './'
    font = 'arial'

# printing the working directory files. One can check you see the same folders and files as in the git webpage.
print(os.listdir(DIRECTORY))

from Build_Model import *

# We declare this function here and not in the
# function-storing python file to modify it easily
# as it can change the printouts of the methods
def printout(V, Stats, model): 
    # printing Stats
    print("R2 = %.2f (+/- %.2f) Constraint = %.2f (+/- %.2f)" % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    Vout = tf.convert_to_tensor(np.float32(model.Y))
    Loss_norm, dLoss = Loss_Vout(V, model.Pout, Vout)
    print('Loss Targets', np.mean(Loss_norm))
    Loss_norm, dLoss = Loss_SV(V, model.S)
    print('Loss SV', np.mean(Loss_norm))
    Vin = tf.convert_to_tensor(np.float32(model.X))
    Pin = tf.convert_to_tensor(np.float32(model.Pin))
    if Vin.shape[1] == model.S.shape[1]: # special case
        Vin  = tf.linalg.matmul(Vin, tf.transpose(Pin), b_is_sparse=True)
    Loss_norm, dLoss = Loss_Vin(V, model.Pin, Vin, model.mediumbound)
    print('Loss Vin bound', np.mean(Loss_norm))
    Loss_norm, dLoss = Loss_Vpos(V, model)
    print('Loss V positive', np.mean(Loss_norm))

['README.md', 'Duplicate_Model.ipynb', 'Build_Model_Dense.ipynb', 'Build_Dataset.py', 'Dataset_experimental', '.ipynb_checkpoints', '.git', 'Build_Experimental.ipynb', 'Reservoir', 'Dataset_model', 'Figures.ipynb', 'Result', 'Figures', '.gitignore', 'Duplicate_Model.py', 'LICENSE', 'Build_Dataset.ipynb', 'Dataset_input', '__pycache__', 'Build_Experimental.py', 'old', 'environment_amn.yml', 'Build_Model.py', 'Build_Model.ipynb', '.DS_Store']


## (1) Mechanistic Model: examples with non trainable mechanistic model with FBA simulated training sets or experimental datasets

In [2]:
# Run Mechanistic model (no training) QP (quadratic program) or LP (linear program)
# using tiny5 simulation training sets with EB (or UB) bounds

# What you can change
seed = 1
np.random.seed(seed=seed)  
trainname = 'tiny5_EB'
write_loss = False # Writes an output file containing loss history mean and std over timesteps
write_targets = False # Writes an output file containing true and predicted targets
# End of What you can change

# Create model and run GD
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['biomass'], 
              model_type = 'MM_LP',
              timestep = int(1.0e3), learn_rate = 1.0, decay_rate = 0.5)
model.printout()

loss_outname = DIRECTORY + "Result/" + model.model_type + "_Loss_" + trainname + ".csv" if write_loss else None
model.loss_outfile = loss_outname
targets_outname = DIRECTORY + "Result/" + model.model_type + "_Targets_" + trainname + ".csv" if write_targets else None
model.targets_outfile = targets_outname

if model.model_type is 'MM_QP':
    Ypred, Stats = MM_QP(model, verbose=True)
if model.model_type is 'MM_LP':
    Ypred, Stats = MM_LP(model, verbose=True)

# Printing results
printout(Ypred, Stats, model)

./Dataset_model/tiny5_EB.npz


SystemExit: parameter file not found

/home/lfaure/miniconda3/envs/AMN/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
# Run Mechanistic model (no training) QP (quadratic program) or LP (linear program)
# using E. coli core simulation training sets and EB (or UB) bounds

# What you can change
seed = 10
np.random.seed(seed=seed)  
trainname = 'e_coli_core_UB' # the training set file name
size = 10 # number of runs must be lower than the number of element in trainname
timestep = int(1.0e4) # LP 1.0e4 QP 1.0e5
learn_rate = 0.3 # LP 0.3 QP 1.0
decay_rate = 0.9 # only in QP, UB 0.333 EB 0.9
write_loss = False # Writes an output file containing loss history mean and std over timesteps
write_targets = False # Writes an output file containing true and predicted targets
# End of What you can change

# Create model and run GD for X and Y randomly drawn from trainingfile
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ecoli_core_w_GAM'], 
              model_type = 'MM_LP', 
              timestep = timestep, 
              learn_rate = learn_rate, 
              decay_rate = decay_rate)

# Select a random subset of the training set (of specified size)

ID = np.random.choice(model.X.shape[0], size, replace=False)
model.X, model.Y = model.X[ID,:], model.Y[ID,:]
if model.mediumbound == 'UB':
    model.b_ext = model.b_ext[ID,:]
if model.mediumbound == 'EB':
    model.b_int = model.b_int[ID,:]

# Prints a summary of the model before running

model.printout()

# Make names for output file (if 'write_' flag is set to true)

loss_outname = DIRECTORY + "Result/" + model.model_type + "_Loss_" + \
trainname + ".csv" if write_loss else None
model.loss_outfile = loss_outname
targets_outname = DIRECTORY + "Result/" + model.model_type + "_Targets_" + \
trainname + ".csv" if write_targets else None
model.targets_outfile = targets_outname

# Runs the appropriate method

if model.model_type is 'MM_QP':
    Ypred, Stats = MM_QP(model, verbose=True)
if model.model_type is 'MM_LP':
    Ypred, Stats = MM_LP(model, verbose=True)

# Printing results
printout(Ypred, Stats, model)

training file: ./Dataset_model/e_coli_core_UB
model type: MM_LP
model scaler: 0.0
model input dim: 20
model output dim: 1
model medium bound: UB
timestep: 10000
training set size (10, 20) (10, 1)
LP-Loss 1 0.25261024 0.06849076
LP-Loss 10 0.2294474 0.06633821
LP-Loss 100 0.0808966 0.03674851
LP-Loss 1000 0.00023186656 0.0002091549
LP-Loss 2000 3.8031656e-05 2.6454396e-05
LP-Loss 3000 1.6315684e-05 1.6629287e-05
LP-Loss 4000 9.442818e-06 1.449519e-05
LP-Loss 5000 7.2193407e-06 1.2035626e-05
LP-Loss 6000 5.5323576e-06 9.809663e-06
LP-Loss 7000 4.2318406e-06 7.880076e-06
LP-Loss 8000 3.2548737e-06 6.2515983e-06
LP-Loss 9000 2.3623904e-06 4.757077e-06
LP-Loss 10000 1.6831349e-06 3.5644443e-06
AMN output shapes for PoutV, SV, PinV, Vpos, V, outputs (10, 1) (10, 1) (10, 1) (10, 1) (10, 154) (10, 312)
R2 = 1.00 (+/- 0.00) Constraint = 0.00 (+/- 0.00)
Loss Targets 0.001244092
Loss SV 8.0880665e-07
Loss Vin bound 1.0041353e-05
Loss V positive 2.924077e-06


In [ ]:
# Run Mechanistic model (no training) QP (quadratic program) or LP (linear program)
# using E. coli core simulation training sets and EB (or UB) bounds

# What you can change
seed = 10
np.random.seed(seed=seed)  
trainname = 'e_coli_core_EB' # the training set file name
size = 10 # number of runs must be lower than the number of element in trainname
timestep = int(1.0e6) # LP 1.0e4 QP 1.0e5
learn_rate = 1.0 # LP 0.3 QP 1.0
decay_rate = 0.9 # only in QP, UB 0.333 EB 0.9
write_loss = False # Writes an output file containing loss history mean and std over timesteps
write_targets = False # Writes an output file containing true and predicted targets
# End of What you can change

# Create model and run GD for X and Y randomly drawn from trainingfile
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ecoli_core_w_GAM'], 
              model_type = 'MM_QP', 
              timestep = timestep, 
              learn_rate = learn_rate, 
              decay_rate = decay_rate)

# Select a random subset of the training set (of specified size)
ID = np.random.choice(model.X.shape[0], size, replace=False)
model.X, model.Y = model.X[ID,:], model.Y[ID,:]
if model.mediumbound == 'UB':
    model.b_ext = model.b_ext[ID,:]
if model.mediumbound == 'EB':
    model.b_int = model.b_int[ID,:]

# Prints a summary of the model before running
model.printout()

# Make names for output file (if 'write_' flag is set to true)
loss_outname = DIRECTORY + "Result/" + model.model_type + "_Loss_" \
+ trainname + ".csv" if write_loss else None
model.loss_outfile = loss_outname
targets_outname = DIRECTORY + "Result/" + model.model_type + "_Targets_" \
+ trainname + ".csv" if write_targets else None
model.targets_outfile = targets_outname

# Runs the appropriate method
if model.model_type is 'MM_QP':
    Ypred, Stats = MM_QP(model, loss_outfile=loss_outname, 
                         targets_outfile= targets_outname,
                         verbose=True)
if model.model_type is 'MM_LP':
    Ypred, Stats = MM_LP(model, loss_outfile=loss_outname, 
                         targets_outfile= targets_outname,
                         verbose=True)
# Printing results
printout(Ypred, Stats, model)

In [ ]:
# Run Mechanistic model (no training) QP (quadratic program) or LP (linear program)
# using E. coli core simulation training sets
# with initial X close to solution Y and EB (or UB) bounds

# What you can change
seed = 10
np.random.seed(seed=seed)  
trainname = 'e_coli_core_UB' # the training set file name (EB or UB)
size = 10 # number of runs must be lower than the number of element in trainname
timestep = int(1.0e4) # LP 1.0e4 QP 1.0e5
learn_rate = 1.0 # LP 0.3 QP 1.0
decay_rate = 0.9 # only in QP, UB 0.333 EB 0.9 # try 0.5
write_loss = False # Writes an output file containing loss history mean and std over timesteps
write_targets = False # Writes an output file containing true and predicted targets
model_type = 'MM_QP'
# End of What you can change

# Draw X and Y randomly drawn from trainingfile
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ecoli_core_w_GAM'], 
              model_type = model_type, 
              timestep = timestep,
              learn_rate = learn_rate,
              decay_rate = decay_rate)
ID = np.random.choice(model.X.shape[0], size, replace=False)
model.X, model.Y, model.Yall = model.X[ID,:], model.Y[ID,:],  model.Yall[ID,:]
if model.mediumbound == 'UB':
    model.b_ext = model.b_ext[ID,:]
if model.mediumbound == 'EB':
    model.b_int = model.b_int[ID,:]
model.printout()

# Create model and run GD X = normal distribution around solution
Ypred, Stats = {}, {}
noise_set = [0.5, 0.1, 0.01, 0.005, 0.001, 0.0005]
for noise in noise_set:
    
    loss_outname = DIRECTORY + "Result/" + str(noise) + "_noise_" + \
    model.model_type + "_Loss_" + trainname + ".csv" if write_loss else None
    model.loss_outfile = loss_outname
    targets_outname = DIRECTORY + "Result/" + str(noise) + "_noise_" \
    + model.model_type + "_Targets_" + trainname + ".csv" if write_targets else None
    model.targets_outfile = targets_outname
    
    
    model.X = np.random.normal(model.Yall, noise*model.Yall)
    print('noise', noise)
    if model.model_type is 'MM_QP':
        Ypred[noise], Stats[noise] = MM_QP(model, verbose=True)
    if model.model_type is 'MM_LP':
        Ypred[noise], Stats[noise] = MM_LP(model, verbose=True)
    
    printout(Ypred[noise], Stats[noise], model)

training file: ./Dataset_model/e_coli_core_UB
model type: MM_QP
model input dim: 20
model output dim: 1
model medium bound: UB
timestep: 10000
training set size (10, 20) (10, 1)
noise 0.5
GD-Loss 1 2.3497357 0.58461136
GD-Loss 10 2.1286168 0.66098446
GD-Loss 100 0.68807036 0.15949886
GD-Loss 1000 0.17094067 0.048883222
GD-Loss 2000 0.112243116 0.03465901
GD-Loss 3000 0.08686058 0.028503802
GD-Loss 4000 0.071525335 0.024566654
GD-Loss 5000 0.061294593 0.021584878
GD-Loss 6000 0.05490441 0.019376915
GD-Loss 7000 0.048432 0.01873374
GD-Loss 8000 0.044350315 0.016383402
GD-Loss 9000 0.040097177 0.015305694
GD-Loss 10000 0.037610233 0.014153577
AMN output shapes for PoutV, SV, PinV, Vpos, V, outputs (10, 1) (10, 1) (10, 1) (10, 1) (10, 154) (10, 158)
R2 = 1.00 (+/- 0.00) Constraint = 0.10 (+/- 0.03)
Loss Targets 0.001111585
Loss SV 0.034170542
Loss Vin bound 0.20221159
Loss V positive 0.0014855479
noise 0.1
GD-Loss 1 0.47501326 0.122467846
GD-Loss 10 0.38130006 0.13565588
GD-Loss 100 0.1461

## (2) Neural model: examples of trainable ANN model (neural only) with FBA simulated training set or experimental datasets

In [1]:
# Run this cell first
import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    DIRECTORY = '/content/drive/My Drive/AMN-Colab/'
    sys.path.append('/content/drive/My Drive/amn')
    ! pip install cobra
    ! pip install silence_tensorflow
else:
    DIRECTORY = './'
print(os.listdir(DIRECTORY))
    
from Build_Model import *

def printout(filename, Stats, model, time): 
    # printing Stats
    print('Stats for %s CPU-time %.4f' % (filename, time))
    print('R2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    print('Q2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.test_objective[0], Stats.test_objective[1],
           Stats.test_loss[0], Stats.test_loss[1]))

['Result', '.DS_Store', 'Dataset_model', 'Reservoir', 'old', '__pycache__', 'README.md', '.gitignore', 'Build_Dataset.ipynb', 'Build_Model.py', '.ipynb_checkpoints', 'Dataset_input', 'Build_Dataset.py', '.git', 'Build_Model.ipynb']


### FBA simulated training set

In [21]:
# Create, train and evaluate ANN models with FBA simulated training set for E. coli core

# What you can change 
seed = 10
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'e_coli_core_UB'
# End of What you can change

# Create model
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ecoli_core_w_GAM'], 
              model_type = 'ANN_Dense',
              n_hidden = 1, hidden_dim = 50, 
              epochs = 500, xfold = 5)
ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0) 
model.printout()

# Train and evaluate
reservoirname = trainname +'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y = Xtrain, Ytrain
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
reservoir.save(reservoirfile)
reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    X, Y = model_input(reservoir,verbose=False)
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)
"""
Stats for e_coli_core_UB_ANN_Dense CPU-time 149.2822
R2 = 0.9587 (+/- 0.0329) Constraint = -1.0000 (+/- 0.0000)
Q2 = 0.9582 (+/- 0.0327) Constraint = -1.0000 (+/- 0.0000)
Stats for Test set CPU-time 0.0347
R2 = 0.9104 (+/- 0.0000) Constraint = -1.0000 (+/- 0.0000)
"""

training file: ./Dataset_model/e_coli_core_UB
model type: ANN_Dense
model scaler: 0.0
model input dim: 20
model output dim: 1
model medium bound: UB
timestep: 0
training set size (1000, 20) (1000, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu
training epochs: 500
training regression: True
training learn rate: 0.001
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
nbr parameters: 1101
6/6 [==============================] - 0s 384us/step
train = 0.99 test = 0.99 loss-train = -1.000000 loss-test = -1.000000 iter=0
nbr parameters: 1101
6/6 [==============================] - 0s 443us/step
train = 0.93 test = 0.93 loss-train = -1.000000 loss-test = -1.000000 iter=0
nbr parameters: 1101
6/6 [==============================] - 0s 362us/step
train = 0.98 test = 0.98 loss-train = -1.000000 loss-test = -1.000000 iter=0
nbr parameters: 1101
6/6 [==============================] - 0s 323us/step
train = 

In [3]:
# Create, train and evaluate ANN models with FBA simulated training set for E. coli core

# What you can change 
seed = 10
np.random.seed(seed=seed)  
trainname = 'e_coli_core_UB_5000'
# End of What you can change

# Create model
trainingfile = DIRECTORY+'Dataset_model/'+trainname
cobramodel = cobra.io.read_sbml_model(trainingfile+'.xml')
OBJ = [r.id for r in cobramodel.reactions]
parameter = TrainingSet()
parameter.load(trainingfile)
Y, i = {}, 0
for obj in OBJ:
    objective = [obj]
    print(objective)
    model = Neural_Model(trainingfile = trainingfile, 
                         objective=objective, 
                         model_type = 'ANN_Dense',
                         n_hidden = 1, hidden_dim = 50, 
                         epochs = 500, xfold = 5)
        
    # Train and evaluate
    start_time = time.time()
    try:
        reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
    except:
        pred = 0 * parameter.Y # zero vector
    delta_time = time.time() - start_time

    # Printing cross-validation results
    print(objective,'----------------------------------------------------------------------', i)
    printout(objective, stats, reservoir, delta_time)
    Y[i] = pred[:,0]
    i = i+1

# Collate all predicted Y and get stats and constraints
Y = np.transpose(np.asarray(list(Y.values())))
print(Y.shape, parameter.Y.shape)
print('Q2=', r2_score(parameter.Y, Y, multioutput='variance_weighted'))
X = tf.convert_to_tensor(np.float32(model.X)) # Loss computed of tf tensors
Y = tf.convert_to_tensor(np.float32(Y))
L2, _ = Loss_SV(Y, model.S)
L2 = np.mean(L2.numpy())
print('Loss_SV =', L2)
L3, _ = Loss_Vin(Y, model.Pin, X, 'UB')
L3 = np.mean(L3.numpy())
print('Loss_Vin =', L3)
L = (L2+L3)/2
print('Constraints =', L)

"""
size 10
hidden_dim = 50 no scaler
(10, 154) (10, 154)
Q2= -0.07538283773845325
Loss_SV 0.48731223
Loss_Vin 0.2028226
Loss_cst 0.34506741166114807
hidden_dim = 50 with scaler : no better
Q2= -0.04873539601415515
Loss_SV = 0.45491654
Loss_Vin = 0.20206484
Constraints = 0.328490674495697

size 100
hidden_dim = 50 no scaler
(100, 154) (100, 154)
Q2= 0.8483248084564434
Loss_SV 0.36500132
Loss_Vin 0.019197455
Loss_cst 0.19209939241409302
hidden_dim = 100 : no better
(100, 154) (100, 154)
Q2= 0.7824626824411614
Loss_SV 0.46771127
Loss_Vin 0.02226068
Loss_cst 0.24498596787452698

size 500 no scaler

size 1000 no scaler
(1000, 154) (1000, 154)
Q2= 0.9507007593125759
Loss_SV 0.19790417
Loss_Vin 0.013121797
Loss_cst 0.10551298409700394

size 10000 no scaler

"""

['PFK']
nbr parameters: 1101
32/32 [==============================] - 0s 266us/step
train = 1.00 test = 1.00 loss-train = -1.000000 loss-test = -1.000000 iter=0
nbr parameters: 1101
32/32 [==============================] - 0s 247us/step
train = 1.00 test = 1.00 loss-train = -1.000000 loss-test = -1.000000 iter=0
nbr parameters: 1101
32/32 [==============================] - 0s 285us/step
train = 1.00 test = 1.00 loss-train = -1.000000 loss-test = -1.000000 iter=0
nbr parameters: 1101
32/32 [==============================] - 0s 252us/step
train = 1.00 test = 1.00 loss-train = -1.000000 loss-test = -1.000000 iter=0
nbr parameters: 1101
32/32 [==============================] - 0s 271us/step
train = 1.00 test = 1.00 loss-train = -1.000000 loss-test = -1.000000 iter=0
157/157 [==============================] - 0s 225us/step
['PFK'] ---------------------------------------------------------------------- 0
Stats for ['PFK'] CPU-time 720.0392
R2 = 0.9976 (+/- 0.0007) Constraint = -1.0000 (+/- 0.

'\nsize 10\nhidden_dim = 50 no scaler\n(10, 154) (10, 154)\nQ2= -0.07538283773845325\nLoss_SV 0.48731223\nLoss_Vin 0.2028226\nLoss_cst 0.34506741166114807\nhidden_dim = 50 with scaler : no better\nQ2= -0.04873539601415515\nLoss_SV = 0.45491654\nLoss_Vin = 0.20206484\nConstraints = 0.328490674495697\n\nsize 100\nhidden_dim = 50 no scaler\n(100, 154) (100, 154)\nQ2= 0.8483248084564434\nLoss_SV 0.36500132\nLoss_Vin 0.019197455\nLoss_cst 0.19209939241409302\nhidden_dim = 100 : no better\n(100, 154) (100, 154)\nQ2= 0.7824626824411614\nLoss_SV 0.46771127\nLoss_Vin 0.02226068\nLoss_cst 0.24498596787452698\n\nsize 500 no scaler\n\nsize 1000 no scaler\n(1000, 154) (1000, 154)\nQ2= 0.9507007593125759\nLoss_SV 0.19790417\nLoss_Vin 0.013121797\nLoss_cst 0.10551298409700394\n\nsize 10000 no scaler\n\n'

## (3) Hybrid model: examples of trainable AMN model (neural and mechanistic) with FBA simulated training sets or experimental datasets

In [1]:
# Run this cell first
import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    DIRECTORY = '/content/drive/My Drive/AMN-Colab/'
    sys.path.append('/content/drive/My Drive/amn')
    ! pip install cobra
    ! pip install silence_tensorflow
else:
    DIRECTORY = './'
print(os.listdir(DIRECTORY))
    
from Build_Model import *

def printout(filename, Stats, model, time): 
    # printing Stats
    print('Stats for %s CPU-time %.4f' % (filename, time))
    print('R2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    print('Q2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.test_objective[0], Stats.test_objective[1],
           Stats.test_loss[0], Stats.test_loss[1]))

['Result', '.DS_Store', 'Dataset_model', 'Reservoir', 'old', '__pycache__', 'README.md', '.gitignore', 'Build_Dataset.ipynb', 'Build_Model.py', '.ipynb_checkpoints', 'Dataset_input', 'Build_Dataset.py', '.git', 'Build_Model.ipynb']


### AMN LP or QP Neural + Mechanistic programs

In [2]:
# Create, train and evaluate AMN_QP o models with FBA simulated training set for E. coli core
# with EB or UB with a mechanistic layer

# What you can change 
seed = 10
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'e_coli_core_UB' # can change EB by UB
timestep = 4
# End of What you can change
  
# Create model 90% for training 10% for testing
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
                     objective=['BIOMASS_Ecoli_core_w_GAM'],  
                     model_type='AMN_QP', 
                     timestep = timestep, learn_rate=0.01,
                     scaler=True,
                     n_hidden = 1, hidden_dim = 50,
                     epochs=500, xfold=5,
                     verbose=True)
ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y = Xtrain, Ytrain
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
reservoir.save(reservoirfile)
reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    X, Y = model_input(reservoir, verbose=False)
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)
    

number of reactions:  154 154
number of metabolites:  72
filtered measurements size:  1
training file: ./Dataset_model/e_coli_core_UB
model type: AMN_QP
model scaler: 1.0
model input dim: 20
model output dim: 1
model medium bound: UB
timestep: 4
training set size (1000, 20) (1000, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu
gradient learn rate: 0.01
gradient decay rate: 0.9
training epochs: 500
training regression: True
training learn rate: 0.001
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
nbr parameters: 8904
6/6 [==============================] - 0s 2ms/step
train = 0.97 test = 0.97 loss-train = 0.002241 loss-test = 0.002233 iter=0
nbr parameters: 8904
6/6 [==============================] - 0s 2ms/step
train = 0.99 test = 0.99 loss-train = 0.003301 loss-test = 0.003456 iter=0
nbr parameters: 8904
6/6 [==============================] - 0s 2ms/step
train = 0.96 test = 0.96 loss-tra

In [3]:
# Create, train and evaluate AMN_LP models with FBA simulated training set for E. coli core
# with UB with a mechanistic layer

# What you can change 
seed = 10
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'e_coli_core_UB' # can change EB by UB
timestep = 4
# End of What you can change
  
# Create model 90% for training 10% for testing
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
                     objective=['BIOMASS_Ecoli_core_w_GAM'],  
                     model_type='AMN_LP', 
                     timestep = timestep, learn_rate=1.0e-6,
                     scaler=True,
                     n_hidden = 1, hidden_dim = 50,
                     epochs= 500, xfold=5,
                     verbose=True)
ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), 
                      replace=False)
# LP keeps track of boundary fluxes in b_int and b_ext
# and these are different in EB or UB modes
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
btest = model.b_ext[ID,:] if model.mediumbound == 'UB' else model.b_int[ID,:]
bint  = model.b_int if model.mediumbound == 'UB' else model.b_ext
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
btrain = np.delete(model.b_ext, ID, axis=0) if model.mediumbound == 'UB' \
else np.delete(model.b_int, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y, model.b_ext, model.b_int = Xtrain, Ytrain, btrain, bint
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
reservoir.save(reservoirfile)
reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    reservoir.b_ext, reservoir.b_int =  btest, bint
    X, Y = model_input(reservoir,verbose=False)
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)
    

number of reactions:  154 154
number of metabolites:  72
filtered measurements size:  1
training file: ./Dataset_model/e_coli_core_UB
model type: AMN_LP
model scaler: 1.0
model input dim: 20
model output dim: 1
model medium bound: UB
timestep: 4
training set size (1000, 20) (1000, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu
training epochs: 500
training regression: True
training learn rate: 0.001
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
nbr parameters: 25152
6/6 [==============================] - 0s 2ms/step
train = 0.96 test = 0.96 loss-train = 0.002815 loss-test = 0.002802 iter=0
nbr parameters: 25152
6/6 [==============================] - 0s 2ms/step
train = 0.97 test = 0.98 loss-train = 0.002668 loss-test = 0.002781 iter=0
nbr parameters: 25152
6/6 [==============================] - 0s 2ms/step
train = 0.97 test = 0.97 loss-train = 0.002129 loss-test = 0.002127 iter=0
nbr pa

In [10]:
# Create, train and evaluate AMN_QP models with FBA simulated training set for iML1515
# with EB or UB with a mechanistic layer 
# This cell takes several hours to execute

# What you can change 
seed = 1
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'iML1515_UB'
timestep = 4
# End of What you can change
  
# Create model 
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_QP',
              scaler = True,
              timestep = timestep, learn_rate=0.01,
              n_hidden = 1, hidden_dim = 500,
              epochs = 25, xfold = 5, 
              verbose=True) 
ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y = Xtrain, Ytrain
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=2)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
#reservoir.save(reservoirfile)
#reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    X, Y = model_input(reservoir, verbose=False)
    reservoir.model.b_ext = btest
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)
 

number of reactions:  550 550
number of metabolites:  1083
filtered measurements size:  1
training file: ./Dataset_model/iML1515_UB
model type: AMN_QP
model scaler: 1.0
model input dim: 38
model output dim: 1
model medium bound: UB
timestep: 4
training set size (11000, 38) (11000, 1)
nbr hidden layer: 1
hidden layer size: 500
activation function: relu
gradient learn rate: 0.01
gradient decay rate: 0.9
training epochs: 25
training regression: True
training learn rate: 0.001
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
AMN scaler 11.0
QP input shape (9900, 38) (9900, 4)
-------train [   0    1    3 ... 9897 9898 9899]
-------test  [   2    4    6 ... 9883 9891 9893]
Dense layer n_hidden, hidden_dim, output_dim, activation, trainable: 1 500 550 relu True
AMN output shapes for PoutV, SV, PinV, Vpos, V, outputs (None, 1) (None, 1) (None, 1) (None, 1) (None, 550) (None, 1104)
Model: "model_95"
_____________________

In [11]:
# Create, train and evaluate AMN_LP models with FBA simulated training set for iML1515
# with UB with a mechanistic layer 
# This cell takes several hours to execute

# What you can change 
seed = 1
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'iML1515_UB'
timestep = 4
# End of What you can change
  
# Create model 
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_LP',
              scaler = True,
              timestep = timestep, learn_rate=1.0e-6,
              n_hidden = 1, hidden_dim = 250,
              epochs = 25, xfold = 5, 
              verbose=True) 

ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), 
                      replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
btest = model.b_ext[ID,:]
bint = model.b_int
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
btrain = np.delete(model.b_ext, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y, model.b_ext, model.b_int = Xtrain, Ytrain, btrain, bint
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=2)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
#reservoir.save(reservoirfile)
#reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    reservoir.b_ext, reservoir.b_int =  btest, bint
    X, Y = model_input(reservoir,verbose=False)
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)


number of reactions:  550 550
number of metabolites:  1083
filtered measurements size:  1
training file: ./Dataset_model/iML1515_UB
model type: AMN_LP
model scaler: 1.0
model input dim: 38
model output dim: 1
model medium bound: UB
timestep: 4
training set size (11000, 38) (11000, 1)
nbr hidden layer: 1
hidden layer size: 250
activation function: relu
training epochs: 25
training regression: True
training learn rate: 0.001
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
AMN scaler 11.0
LP input shape (9900, 3235) (9900, 4)
-------train [   0    1    3 ... 9897 9898 9899]
-------test  [   2    4    6 ... 9883 9891 9893]
Dense layer n_hidden, hidden_dim, output_dim, activation, trainable: 1 250 550 relu True
Dense layer n_hidden, hidden_dim, output_dim, activation, trainable: 1 250 2716 linear True
AMN output shapes for PoutV, SV, PinV, Vpos, V, outputs (None, 1) (None, 1) (None, 1) (None, 1) (None, 550) (None, 11

### AMN Wt program

In [9]:
# Create, train and evaluate AMN_Wt models with FBA simulated training set for E. coli core
# with UB (not working with M1 chips)

# What you can change 
seed = 10
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'e_coli_core_UB' # can change EB by UB
timestep = 4
# End of What you can change
  
# Create model 90% for training 10% for testing
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
                     objective=['BIOMASS_Ecoli_core_w_GAM'],  
                     model_type='AMN_Wt', 
                     timestep = timestep,
                     n_hidden = 1, hidden_dim = 50,
                     scaler=True,
                     train_rate=1e-2,
                     epochs=500, xfold=5,
                     verbose=True)
ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y = Xtrain, Ytrain
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
#reservoir.save(reservoirfile)
#reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    X, Y = model_input(reservoir, verbose=False)
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)

number of reactions:  154 154
number of metabolites:  72
filtered measurements size:  1
training file: ./Dataset_model/e_coli_core_UB
model type: AMN_Wt
model scaler: 1.0
model input dim: 20
model output dim: 1
model medium bound: UB
timestep: 4
training set size (1000, 20) (1000, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu
training epochs: 500
training regression: True
training learn rate: 0.01
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
nbr parameters: 13262
6/6 [==============================] - 0s 2ms/step
train = 0.99 test = 0.99 loss-train = 0.000919 loss-test = 0.000912 iter=0
nbr parameters: 13262
6/6 [==============================] - 0s 2ms/step
train = 0.99 test = 0.99 loss-train = 0.000933 loss-test = 0.000990 iter=0
nbr parameters: 13262
6/6 [==============================] - 0s 2ms/step
train = 0.99 test = 0.99 loss-train = 0.000908 loss-test = 0.000904 iter=0
nbr par

In [12]:
# Create, train and evaluate AMN_QP models with FBA simulated training set for iML1515
# with EB or UB with a mechanistic layer 
# This cell takes several hours to execute

# What you can change 
seed = 1
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'iML1515_UB'
timestep = 4
# End of What you can change
  
# Create model 
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_Wt',
              scaler = True,
              timestep = timestep, 
              n_hidden = 1, hidden_dim = 500,
              train_rate=1e-2,
              epochs = 100, xfold = 5, 
              verbose=True) 

ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), 
                      replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0) 
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y = Xtrain, Ytrain
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=2)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
#reservoir.save(reservoirfile)
#reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    X, Y = model_input(reservoir,verbose=False)
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)

number of reactions:  550 550
number of metabolites:  1083
filtered measurements size:  1
training file: ./Dataset_model/iML1515_UB
model type: AMN_Wt
model scaler: 1.0
model input dim: 38
model output dim: 1
model medium bound: UB
timestep: 4
training set size (11000, 38) (11000, 1)
nbr hidden layer: 1
hidden layer size: 500
activation function: relu
training epochs: 100
training regression: True
training learn rate: 0.01
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
AMN scaler 11.0
Wt input shape (9900, 4, 38) (9900, 4)
-------train [   0    1    3 ... 9897 9898 9899]
-------test  [   2    4    6 ... 9883 9891 9893]
number of reactions:  550 550
number of metabolites:  1083
filtered measurements size:  1
Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to             

### AMN -QP, -LP or -Wt with experimental training set

In [ ]:
# Create, train and evaluate AMN_QP models on experimental training set with UB 
# Repeat the process with different seeds
# This cell takes several hours to execute

Maxloop, Q2, PRED = 3, [], []

for Nloop in range(Maxloop):
    # What you can change 
    seed = Nloop+1
    np.random.seed(seed=seed)  
    trainname = 'iML1515_EXP_UB' 
    timestep = 4
    # End of What you can change
  
    # Create model 100% for training 0% for testing
    trainingfile = DIRECTORY+'Dataset_model/'+trainname
    model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_QP',
              scaler = True,
              timestep = timestep, learn_rate=0.001,
              n_hidden = 1, hidden_dim = 500,
              #train_rate = 1.0e-2,
              epochs = 1000, xfold = 10, 
              verbose=True) 

    # Train and evaluate
    reservoirname = trainname +'_'+model.model_type
    reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
    start_time = time.time()
    reservoir, pred, stats, _ = train_evaluate_model(model, verbose=True)
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(reservoirname, stats, model, delta_time)
    r2 = r2_score(model.Y, pred[:,0], multioutput='variance_weighted')
    print('Iter', Nloop, 'Collated Q2', r2)
    Q2.append(r2)
    PRED.append(pred[:,0])

# Save in Result folder
Q2, PRED = np.asarray(Q2), np.asarray(PRED)
print('Averaged Q2 = %.4f (+/- %.4f)' % (np.mean(Q2), np.std(Q2)))
filename = DIRECTORY+'Result/'+reservoirname+'_Q2.csv'
np.savetxt(filename, Q2, delimiter=',')
filename = DIRECTORY+'Result/'+reservoirname+'_PRED.csv'
np.savetxt(filename, PRED, delimiter=',')


In [ ]:
# Create, train and evaluate AMN_QP models on experimental training set with UB 
# Repeat the process with different seeds
# This cell takes several hours to execute

Maxloop, Q2, PRED = 3, [], []

for Nloop in range(Maxloop):
    # What you can change 
    seed = Nloop+1
    np.random.seed(seed=seed)  
    trainname = 'iML1515_EXP_UB' 
    timestep = 4
    # End of What you can change
  
    # Create model 100% for training 0% for testing
    trainingfile = DIRECTORY+'Dataset_model/'+trainname
    model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_LP',
              scaler = True,
              timestep = timestep, learn_rate=0.001,
              n_hidden = 1, hidden_dim = 500,
              #train_rate = 1.0e-2,
              epochs = 1000, xfold = 10, 
              verbose=True) 

    # Train and evaluate
    reservoirname = trainname +'_'+model.model_type
    reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
    start_time = time.time()
    reservoir, pred, stats, _ = train_evaluate_model(model, verbose=2)
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(reservoirname, stats, model, delta_time)
    r2 = r2_score(model.Y, pred[:,0], multioutput='variance_weighted')
    print('Iter', Nloop, 'Collated Q2', r2)
    Q2.append(r2)
    PRED.append(pred[:,0])

# Save in Result folder
Q2, PRED = np.asarray(Q2), np.asarray(PRED)
print('Averaged Q2 = %.4f (+/- %.4f)' % (np.mean(Q2), np.std(Q2)))
filename = DIRECTORY+'Result/'+reservoirname+'_Q2.csv'
np.savetxt(filename, Q2, delimiter=',')
filename = DIRECTORY+'Result/'+reservoirname+'_PRED.csv'
np.savetxt(filename, PRED, delimiter=',')


In [ ]:
# Create, train and evaluate AMN_Wt models on experimental training set with UB 
# Repeat the process with different seeds
# This cell takes several hours to execute

Maxloop, Q2, PRED = 3, [], []

for Nloop in range(Maxloop):
    # What you can change 
    seed = Nloop+1
    np.random.seed(seed=seed)  
    trainname = 'iML1515_EXP_UB' 
    timestep = 4
    # End of What you can change
  
    # Create model 100% for training 0% for testing
    trainingfile = DIRECTORY+'Dataset_model/'+trainname
    model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_Wt',
              scaler = True,
              timestep = timestep,
              n_hidden = 1, hidden_dim = 500,
              #train_rate = 1.0e-2,
              epochs = 1000, xfold = 10, 
              verbose=True) 

    # Train and evaluate
    reservoirname = trainname +'_'+model.model_type
    reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
    start_time = time.time()
    reservoir, pred, stats, _ = train_evaluate_model(model, verbose=True)
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(reservoirname, stats, model, delta_time)
    r2 = r2_score(model.Y, pred[:,0], multioutput='variance_weighted')
    print('Iter', Nloop, 'Collated Q2', r2)
    Q2.append(r2)
    PRED.append(pred[:,0])

# Save in Result folder
Q2, PRED = np.asarray(Q2), np.asarray(PRED)
print('Averaged Q2 = %.4f (+/- %.4f)' % (np.mean(Q2), np.std(Q2)))
filename = DIRECTORY+'Result/'+reservoirname+'_Q2.csv'
np.savetxt(filename, Q2, delimiter=',')
filename = DIRECTORY+'Result/'+reservoirname+'_PRED.csv'
np.savetxt(filename, PRED, delimiter=',')


## (4) Reservoir computing with experimental data set

In [2]:
# Run this cell first
import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    DIRECTORY = '/content/drive/My Drive/AMN-Colab/'
    sys.path.append('/content/drive/My Drive/amn')
    ! pip install cobra
    ! pip install silence_tensorflow
else:
    DIRECTORY = './'
print(os.listdir(DIRECTORY))
    
from Build_Model import *

def printout(filename, Stats, model, time): 
    # printing Stats
    print('Stats for %s CPU-time %.4f' % (filename, time))
    print('R2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    print('Q2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.test_objective[0], Stats.test_objective[1],
           Stats.test_loss[0], Stats.test_loss[1]))

['Result', '.DS_Store', 'Dataset_model', 'Reservoir', 'old', '__pycache__', 'README.md', '.gitignore', 'Build_Dataset.ipynb', 'Build_Model.py', '.ipynb_checkpoints', 'Dataset_input', 'Build_Dataset.py', '.git', 'Build_Model.ipynb']


In [6]:
# Create, train and evaluate RC models on experimental training set 
# Repeat the process with different seeds
# This cell takes several hours to execute

seed = 10
Maxloop, Q2, PRED = 3, [], []  
for Nloop in range(Maxloop):
    # What you can change 
    seed = Nloop
    np.random.seed(seed=seed)  
    trainname = 'iML1515_EXP'
    reservoirname = 'iML1515_UB_AMN_QP'
    # End of What you can change
  
    # Create model 
    trainingfile = DIRECTORY+'Dataset_input/'+trainname
    reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
    X, Y = read_XY(trainingfile, 38)
    model = RC_Model(reservoirfile = reservoirfile, X=X, Y=Y, 
                n_hidden_prior = 1, hidden_dim_prior = 500, activation_prior = 'relu',
                epochs = 1000, train_rate = 1.0e-4, xfold = 0,
                verbose=True) 
    model.printout()

    # Train and evaluate
    start_time = time.time()
    reservoir, pred, stats, _ = train_evaluate_model(model, verbose=True)
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(reservoirname, stats, model, delta_time)
    r2 = r2_score(model.Y, pred[:,0], multioutput='variance_weighted')
    print('Iter', Nloop, 'Collated Q2', r2)
    Q2.append(r2)
    PRED.append(pred)

# Some printings and savings
Q2, PRED = np.asarray(Q2), np.asarray(PRED)
print(PRED.shape)
obj = PRED[:,:,0]
print('Averaged Q2 = %.4f (+/- %.4f)' % (np.mean(Q2), np.std(Q2)))
filename = DIRECTORY+'Result/'+reservoirname+'_'\
+str(model.model_type)+'_Q2.csv'
np.savetxt(filename, Q2, delimiter=',')
filename = DIRECTORY+'Result/'+reservoirname+'_'\
+str(model.model_type)+'_PRED.csv'
np.savetxt(filename, obj, delimiter=',')

# Saving input / output of the first prediction for Cobra runs
pred, obj = PRED[0], pred[:,0]
Loss_SV, Loss_Vin, Loss_Vpos = pred[:,1], pred[:,2], pred[:,3]
print('Loss SV average %.4f max %.4f' % (np.mean(Loss_SV), np.max(Loss_SV)))
print('Loss Vin average %.4f max %.4f' % (np.mean(Loss_Vin), np.max(Loss_Vin)))
print('Loss Vpos average %.4f max %.4f' % (np.mean(Loss_Vpos), np.max(Loss_Vpos)))
V = pred[:, 4:4+model.S.shape[1]]
Vin = pred[:, 4+model.S.shape[1]:]
Vin = model.res.scaler * Vin # rescale back for Cobra
XY = np.concatenate((Vin, Y), axis=1)
filename = DIRECTORY+'Result/'+reservoirname+'_'\
+str(model.model_type)+'_solution_for_Cobra.csv'
np.savetxt(filename, XY, delimiter=',')

number of reactions:  550 550
number of metabolites:  1083
filtered measurements size:  1
RC reservoir file: ./Reservoir/iML1515_UB_AMN_QP
RC model type: RC_AMN
RC scaler: 0.0
RC model input dim: 38
RC model output dim: 1
RC model medium bound: UB
training set size (110, 38) (110, 1)
reservoir S, Pin, Pout matrices (1083, 550) (38, 550) (1, 550)
RC training epochs: 1000
RC training regression: True
RC training learn rate: 0.0001
RC training dropout: 0.25
RC training batch size: 5
RC training validation iter: 0
RC training xfold: 0
RC training early stopping: False
--------prior network --------
training file: None
model type: ANN_Dense
model scaler: 0.0
model input dim: 10
model output dim: 10
model medium bound: 
timestep: 0
no training set provided
nbr hidden layer: 1
hidden layer size: 500
activation function: relu
--------reservoir network-----
training file: ./Dataset_model/iML1515_UB
model type: AMN_QP
model scaler: 11.0
model input dim: 38
model output dim: 1104
model medium bou